# Tahap 1: Data Preprocessing & Cleaning

In [ ]:
import pandas as pd
import geopandas as gpd
import glob
import os

print("Memulai Tahap 1: Preprocessing dan Ekstraksi Gempa Darat")

# --- Bagian 1: Memuat Semua Data Mentah ---
geojson_file = "Batas Provinsi 50m.geojson"
if not os.path.exists(geojson_file):
    raise FileNotFoundError(f"File batas provinsi '{geojson_file}' tidak ditemukan.")

gdf_provinsi = gpd.read_file(geojson_file)
print("Data batas provinsi berhasil dimuat.")

csv_files = glob.glob("*.csv")
csv_files = [f for f in csv_files if 'darat' not in f and 'enriched' not in f and 'bersih' not in f]
if not csv_files:
    raise FileNotFoundError("Tidak ada file CSV data gempa yang ditemukan.")

list_of_dfs = [pd.read_csv(file) for file in csv_files]
df_raw = pd.concat(list_of_dfs, ignore_index=True)
print(f"{len(csv_files)} file CSV gempa berhasil dimuat. Total baris mentah: {len(df_raw)}")

# --- Bagian 2: Pembersihan Data ---
df_raw.drop_duplicates(inplace=True)
print(f"Menghapus baris duplikat.")

df_raw['time'] = pd.to_datetime(df_raw['time'], errors='coerce')
numeric_cols = ['latitude', 'longitude', 'depth', 'mag']
for col in numeric_cols:
    df_raw[col] = pd.to_numeric(df_raw[col], errors='coerce')

critical_cols = ['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'type', 'status']
df_raw.dropna(subset=critical_cols, inplace=True)
print(f"Menghapus baris dengan nilai kritis yang kosong.")

df_clean = df_raw[
    (df_raw['type'] == 'earthquake') &
    (df_raw['status'] == 'reviewed')
].copy()
print(f"Memfilter data. Sisa {len(df_clean)} baris gempa yang terverifikasi.")

# --- Bagian 3: Analisis Spasial (Menentukan Gempa Darat) ---
gdf_gempa = gpd.GeoDataFrame(
    df_clean,
    geometry=gpd.points_from_xy(df_clean.longitude, df_clean.latitude),
    crs="EPSG:4326"
)
gdf_provinsi = gdf_provinsi.to_crs(gdf_gempa.crs)

prov_col_name = next((col for col in ['PROVINSI', 'NAMOBJ'] if col in gdf_provinsi.columns), None)
if not prov_col_name:
    raise ValueError("Kolom nama provinsi (PROVINSI/NAMOBJ) tidak ditemukan di file GeoJSON.")

gempa_darat = gpd.sjoin(gdf_gempa, gdf_provinsi, how="inner", predicate="within")
print(f"Ditemukan {len(gempa_darat)} gempa darat.")

# --- Bagian 4: Finalisasi dan Penyimpanan ---
gempa_darat = gempa_darat.rename(columns={prov_col_name: 'provinsi'})

kolom_final = ['time', 'latitude', 'longitude', 'depth', 'mag', 'magType', 'place', 'provinsi']

# Membuat salinan eksplisit untuk diolah lebih lanjut
df_final = gempa_darat[kolom_final].copy()

# PERBAIKAN: Hindari 'inplace=True' dengan menugaskan kembali hasilnya
df_final = df_final.sort_values(by='time', ascending=False)
print("Finalisasi data dan memilih kolom...")

# Simpan hasil ke file baru
output_csv = 'data_gempa_darat.csv'
df_final.to_csv(output_csv, index=False)

print("\n--- TAHAP 1 SELESAI ---")
print(f"Data akhir berisi {len(df_final)} baris gempa darat.")
print(f"Hasil disimpan ke: {output_csv}")

# Tahap 2: Exploratory Data Analysis (EDA)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter
import seaborn as sns
import geopandas as gpd
import folium

# --- Analisis 1: Pareto Provinsi ---
print("Memulai Analisis 1: Pareto Provinsi")
df_darat = pd.read_csv('data_gempa_darat.csv')

provinsi_counts = df_darat['provinsi'].value_counts()
df_pareto = pd.DataFrame({'jumlah': provinsi_counts})
df_pareto = df_pareto.sort_values(by='jumlah', ascending=False)
df_pareto['persen_kumulatif'] = (df_pareto['jumlah'].cumsum() / df_pareto['jumlah'].sum()) * 100

fig, ax = plt.subplots(figsize=(12, 7))
ax.bar(df_pareto.index[:15], df_pareto['jumlah'][:15], color="C0")
ax.set_ylabel("Jumlah Total Gempa Darat")
plt.xticks(rotation=75)
plt.title("Analisis Pareto: 15 Provinsi dengan Gempa Darat Terbanyak")

ax2 = ax.twinx()
ax2.plot(df_pareto.index[:15], df_pareto['persen_kumulatif'][:15], color="C1", marker="o", ms=5)
ax2.yaxis.set_major_formatter(PercentFormatter())
ax2.set_ylabel("Persentase Kumulatif")

plt.tight_layout()
plt.show()

# --- Analisis 2: Tren Waktu dan Distribusi ---
print("Memulai Analisis 2: Tren Waktu dan Distribusi")

df_darat['time'] = pd.to_datetime(df_darat['time'], format='ISO8601')

sns.set_style("whitegrid")
plt.figure(figsize=(18, 5))

# Grafik 1: Jumlah gempa darat per tahun
plt.subplot(1, 3, 1)
df_darat['year'] = df_darat['time'].dt.year
# PERBAIKAN: Menambahkan hue dan legend=False untuk menghilangkan warning
sns.countplot(y=df_darat['year'], hue=df_darat['year'], palette="viridis", order=df_darat['year'].value_counts().index, legend=False)
plt.title('Jumlah Gempa Darat per Tahun')
plt.xlabel('Jumlah Kejadian')
plt.ylabel('Tahun')

# Grafik 2: Distribusi Magnitudo
plt.subplot(1, 3, 2)
sns.histplot(df_darat['mag'], kde=True, bins=20, color='royalblue')
plt.title('Distribusi Magnitudo Gempa Darat')
plt.xlabel('Magnitudo')
plt.ylabel('Frekuensi')

# Grafik 3: Distribusi Kedalaman
plt.subplot(1, 3, 3)
sns.histplot(df_darat[df_darat['depth'] <= 300]['depth'], kde=True, bins=25, color='darkorange')
plt.title('Distribusi Kedalaman Gempa Darat (km)')
plt.xlabel('Kedalaman (km)')
plt.ylabel('Frekuensi')

plt.tight_layout()
plt.show()

# --- Analisis 3:  Peta Jumlah Kejadian, Magnitudo dan Tahun Tertinggi ---
print("Memulai Analisis 3: Peta Jumlah Kejadian, Magnitudo dan Tahun Tertinggi")

# Muat data yang sudah diproses
df_darat = pd.read_csv('data_gempa_darat.csv')
geojson_file = 'Batas Provinsi 50m.geojson'

# PERBAIKAN: Tambahkan format='ISO8601' untuk menangani waktu yang tidak seragam
df_darat['time'] = pd.to_datetime(df_darat['time'], format='ISO8601')

# a. Hitung jumlah gempa per provinsi
provinsi_counts = df_darat.groupby('provinsi').size().reset_index(name='jumlah_gempa')

# b. Cari baris data untuk magnitudo tertinggi di setiap provinsi
idx_max_mag = df_darat.groupby('provinsi')['mag'].idxmax()
max_mag_events = df_darat.loc[idx_max_mag].copy()

# c. Buat kolom tahun dan pilih kolom yang relevan
max_mag_events['tahun_mag_maks'] = max_mag_events['time'].dt.year
max_mag_events = max_mag_events[['provinsi', 'mag', 'tahun_mag_maks']]
max_mag_events.rename(columns={'mag': 'magnitudo_maks'}, inplace=True)

# d. Gabungkan data jumlah gempa dengan data magnitudo maks
provinsi_stats = pd.merge(provinsi_counts, max_mag_events, on='provinsi', how='left')

# Muat data GeoJSON provinsi dan gabungkan dengan data statistik
gdf_provinsi = gpd.read_file(geojson_file)
prov_col_name = next((col for col in ['PROVINSI', 'NAMOBJ'] if col in gdf_provinsi.columns), None)
peta_data = gdf_provinsi.merge(provinsi_stats, left_on=prov_col_name, right_on='provinsi', how='left')

# Isi data kosong untuk provinsi yang tidak mengalami gempa
peta_data['jumlah_gempa'] = peta_data['jumlah_gempa'].fillna(0).astype(int)
peta_data['magnitudo_maks'] = peta_data['magnitudo_maks'].fillna(0)
peta_data['tahun_mag_maks'] = peta_data['tahun_mag_maks'].fillna('N/A')


# Buat peta dasar
m = folium.Map(location=[-2.5, 118], zoom_start=5)

# 1. Tambahkan layer Choropleth
folium.Choropleth(
    geo_data=peta_data,
    name='choropleth',
    data=peta_data,
    columns=['provinsi', 'jumlah_gempa'],
    key_on=f'feature.properties.{prov_col_name}',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name='Jumlah Kejadian Gempa Darat'
).add_to(m)

# 2. Tambahkan layer GeoJson untuk tooltip interaktif
style_function = lambda x: {'fillColor': 'transparent', 'color': 'transparent', 'weight': 0}
tooltip = folium.features.GeoJsonTooltip(
    fields=['provinsi', 'jumlah_gempa', 'magnitudo_maks', 'tahun_mag_maks'],
    aliases=['Provinsi:', 'Jumlah Gempa:', 'Magnitudo Tertinggi:', 'Tahun Magnitudo Tertinggi:'],
    style=("background-color: white; color: #333333; font-family: arial; font-size: 12px; padding: 10px;"),
    localize=True
)

folium.GeoJson(
    peta_data,
    style_function=style_function,
    tooltip=tooltip
).add_to(m)

folium.LayerControl().add_to(m)

# Tampilkan peta
m


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Muat data gempa darat
df_darat = pd.read_csv('data_gempa_darat.csv')

# Definisikan ambang batas
MAGNITUDE_BERBAHAYA = 6.0
KEDALAMAN_DANGKAL = 70.0 # dalam km

# Filter gempa darat yang kuat dan dangkal
gempa_berbahaya = df_darat[
    (df_darat['mag'] >= MAGNITUDE_BERBAHAYA) & 
    (df_darat['depth'] < KEDALAMAN_DANGKAL)
].copy()

# Hitung jumlah kejadian per provinsi
kejadian_per_provinsi = gempa_berbahaya['provinsi'].value_counts()

print("Provinsi dengan Jumlah Gempa Darat Paling Berbahaya (Magnitudo >= 6.0, Kedalaman < 70 km)")
print("--------------------------------------------------------------------------------------")
print(kejadian_per_provinsi)

# Visualisasi
plt.figure(figsize=(10, 6))

# PERBAIKAN: Menambahkan hue dan legend=False untuk menghilangkan warning
sns.barplot(
    x=kejadian_per_provinsi.values, 
    y=kejadian_per_provinsi.index, 
    hue=kejadian_per_provinsi.index, 
    palette='Reds_r', 
    legend=False
)

plt.title(f'Jumlah Gempa Darat Berpotensi Merusak (M ≥ {MAGNITUDE_BERBAHAYA}, Kedalaman < {KEDALAMAN_DANGKAL} km)')
plt.xlabel('Jumlah Kejadian')
plt.ylabel('Provinsi')
plt.tight_layout()
plt.show()

In [ ]:
%pip install --upgrade nbformat
import pandas as pd
import plotly.express as px

# Muat data gempa darat
df_darat = pd.read_csv('data_gempa_darat.csv')

# Pilih provinsi untuk studi kasus (misalnya 'PAPUA')
PROVINSI_STUDI_KASUS = 'SUMATERA BARAT' 

# Filter data untuk provinsi tersebut
df_provinsi = df_darat[df_darat['provinsi'] == PROVINSI_STUDI_KASUS].copy()

# Pastikan 'time' adalah datetime dan jadikan sebagai index
df_provinsi['time'] = pd.to_datetime(df_provinsi['time'], format='ISO8601')
df_provinsi.set_index('time', inplace=True)

# Gunakan 'ME' untuk resample dan agregasi untuk mendapatkan jumlah dan magnitudo maks
frekuensi_bulanan = df_provinsi['mag'].resample('ME').agg(['size', 'max'])
frekuensi_bulanan.rename(columns={'size': 'jumlah_gempa', 'max': 'magnitudo_maks'}, inplace=True)

# PERBAIKAN: Hindari 'inplace=True' untuk menghilangkan warning
frekuensi_bulanan['magnitudo_maks'] = frekuensi_bulanan['magnitudo_maks'].fillna(0)

# Membuat plot interaktif dengan Plotly Express
fig = px.line(
    frekuensi_bulanan,
    x=frekuensi_bulanan.index,
    y='jumlah_gempa',
    title=f'Frekuensi dan Magnitudo Maks Gempa Darat Bulanan di {PROVINSI_STUDI_KASUS}',
    labels={'index': 'Bulan', 'jumlah_gempa': 'Jumlah Kejadian Gempa'},
    markers=True,
    hover_data={
        'magnitudo_maks': ':.2f' # Menampilkan magnitudo maks di tooltip
    }
)

# Update tampilan tooltip agar lebih informatif
fig.update_traces(
    hovertemplate="<br>".join([
        "<b>Bulan: %{x|%B %Y}</b>",
        "Jumlah Gempa: %{y}",
        "Magnitudo Tertinggi: %{customdata[0]:.2f}",
    ])
)

# Tampilkan plot interaktif
fig.show()

In [ ]:
import pandas as pd
import geopandas as gpd
import folium
import plotly.express as px
import os

print("Memulai Analisis: Peta Interaktif dengan Klasifikasi Marker")

# --- Langkah 1: Persiapan Data (Tidak Berubah) ---

if not os.path.exists('icons'):
    os.makedirs('icons')

df_darat = pd.read_csv('data_gempa_darat.csv')
geojson_file = 'Batas Provinsi 50m.geojson'
gdf_provinsi = gpd.read_file(geojson_file)

def klasifikasi_bmkg(mag):
    if mag < 2.5: return 'Mikro (Tidak Terasa, < 2.5)'
    elif mag <= 5.4: return 'Ringan (Dirasakan, 2.5 - 5.4)'
    elif mag <= 6.0: return 'Sedang (5.5 - 6.0)'
    elif mag <= 6.9: return 'Kuat (6.1 - 6.9)'
    elif mag <= 7.9: return 'Besar (7.0 - 7.9)'
    else: return 'Dahsyat (>= 8.0)'

df_darat['klasifikasi'] = df_darat['mag'].apply(klasifikasi_bmkg)

# --- Langkah 2: Agregasi Data (Tidak Berubah) ---

profil_kerusakan = pd.crosstab(df_darat['provinsi'], df_darat['klasifikasi'])
WARNA_KATEGORI = {
    'Mikro (Tidak Terasa, < 2.5)': '#8FBC8F', # DarkSeaGreen
    'Ringan (Dirasakan, 2.5 - 5.4)': '#87CEEB', # SkyBlue
    'Sedang (5.5 - 6.0)': '#FFD700', # Gold
    'Kuat (6.1 - 6.9)': '#FFA500', # Orange
    'Besar (7.0 - 7.9)': '#DC143C', # Crimson
    'Dahsyat (>= 8.0)': '#8B0000'  # DarkRed
}
KATEGORI_URUTAN = list(WARNA_KATEGORI.keys())

for kat in KATEGORI_URUTAN:
    if kat not in profil_kerusakan.columns:
        profil_kerusakan[kat] = 0
profil_kerusakan['total'] = profil_kerusakan.sum(axis=1)

# --- Langkah 3: Membuat Peta dengan Penyesuaian Baru ---

def style_marker(jumlah_gempa):
    if jumlah_gempa < 100:
        return {'radius': 6, 'color': 'green', 'fill_color': 'green'}
    elif jumlah_gempa < 500:
        return {'radius': 9, 'color': 'orange', 'fill_color': 'orange'}
    else:
        return {'radius': 12, 'color': 'red', 'fill_color': 'red'}

m = folium.Map(location=[-2.5, 118], zoom_start=5, tiles='CartoDB positron')
folium.GeoJson(
    gdf_provinsi,
    style_function=lambda x: {'color': 'black', 'weight': 1, 'fillOpacity': 0.1, 'dashArray': '5, 5'}
).add_to(m)

print("\nMemproses semua provinsi:")
for index, row in profil_kerusakan.iterrows():
    nama_prov = index
    
    provinsi_geom = gdf_provinsi[gdf_provinsi['PROVINSI'] == nama_prov]
    if provinsi_geom.empty:
        provinsi_geom = gdf_provinsi[gdf_provinsi['NAMOBJ'] == nama_prov]
    
    if provinsi_geom.empty:
        continue

    # PERBAIKAN 1: Menghitung centroid pada CRS planar untuk menghilangkan warning
    gdf_planar = provinsi_geom.to_crs("EPSG:3857")
    centroid_planar = gdf_planar.geometry.centroid.iloc[0]
    centroid_latlon = gpd.GeoSeries([centroid_planar], crs="EPSG:3857").to_crs("EPSG:4326").iloc[0]

    data_chart = row.drop('total').reset_index()
    data_chart.columns = ['klasifikasi', 'jumlah']
    
    fig = px.bar(
        data_chart,
        x='klasifikasi',
        y='jumlah',
        color='klasifikasi',
        color_discrete_map=WARNA_KATEGORI,
        category_orders={'klasifikasi': KATEGORI_URUTAN},
        title=f"Detail Gempa di {nama_prov}",
        labels={'jumlah': 'Jumlah Kejadian', 'klasifikasi': 'Klasifikasi Kerusakan'}
    )
    
    # PERBAIKAN 2: Memperbesar ukuran chart dan mengatur skala sumbu Y
    fig.update_layout(
        height=400, # Ukuran tinggi chart diperbesar
        width=600,  # Ukuran lebar chart diperbesar
        showlegend=False,
        yaxis=dict(tickmode='linear', tick0=0, dtick=50)
    )
    
    html_chart = fig.to_html(full_html=False, include_plotlyjs='cdn')
    
    # Sesuaikan ukuran IFrame dan Popup dengan ukuran chart yang baru
    iframe = folium.IFrame(html_chart, width=630, height=430)
    popup = folium.Popup(iframe, max_width=650)
    
    marker_style = style_marker(row['total'])
    folium.CircleMarker(
        location=[centroid_latlon.y, centroid_latlon.x],
        radius=marker_style['radius'],
        color=marker_style['color'],
        fill=True,
        fill_color=marker_style['fill_color'],
        fill_opacity=0.7,
        popup=popup,
        tooltip=f"<b>{nama_prov}</b><br>Total Gempa: {row['total']}<br>Klik untuk detail"
    ).add_to(m)
    print(f"+ Berhasil menambahkan marker untuk {nama_prov}")

legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 180px; height: 100px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:white;
     ">&nbsp; <b>Jumlah Total Gempa</b><br>
     &nbsp; <i style="background:green; border-radius:50%; width:12px; height:12px; float:left; margin-top:4px; margin-right:8px;"></i> < 100 Kejadian<br>
     &nbsp; <i style="background:orange; border-radius:50%; width:12px; height:12px; float:left; margin-top:4px; margin-right:8px;"></i> 100 - 500 Kejadian<br>
     &nbsp; <i style="background:red; border-radius:50%; width:12px; height:12px; float:left; margin-top:4px; margin-right:8px;"></i> > 500 Kejadian<br>
      </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

print("\nPeta interaktif dengan popup chart selesai dibuat.")

# Tampilkan peta
m

In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Muat data gempa darat
df_darat = pd.read_csv('data_gempa_darat.csv')

# Pastikan 'time' adalah datetime dan jadikan sebagai index
df_darat['time'] = pd.to_datetime(df_darat['time'], format='ISO8601')
df_darat.set_index('time', inplace=True)

# Resample data per bulan untuk seluruh Indonesia
# Agregasi untuk mendapatkan jumlah kejadian (size) dan magnitudo maksimum (max)
tren_nasional = df_darat['mag'].resample('ME').agg(['size', 'max'])
tren_nasional.rename(columns={'size': 'jumlah_gempa', 'max': 'magnitudo_maks'}, inplace=True)
tren_nasional['magnitudo_maks'] = tren_nasional['magnitudo_maks'].fillna(0)


# Buat gambar dengan dua sumbu Y
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Tambahkan trace untuk Jumlah Gempa (Bar Chart)
fig.add_trace(
    go.Bar(x=tren_nasional.index, y=tren_nasional['jumlah_gempa'], name='Jumlah Gempa', marker_color='skyblue'),
    secondary_y=False,
)

# Tambahkan trace untuk Magnitudo Maksimum (Line Chart)
fig.add_trace(
    go.Scatter(x=tren_nasional.index, y=tren_nasional['magnitudo_maks'], name='Magnitudo Maksimum', marker_color='crimson'),
    secondary_y=True,
)

# Atur judul dan label
fig.update_layout(
    title_text='Tren Bulanan Gempa Darat Nasional (Jumlah vs Magnitudo Maks)',
    xaxis_title='Tahun'
)

# Atur nama sumbu Y
fig.update_yaxes(title_text="<b>Jumlah Kejadian Gempa</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Magnitudo Maksimum</b>", secondary_y=True)

fig.show()